# Just a file to revise all langgraph topics needed for this project

In [7]:
from langchain_ollama import ChatOllama
model = ChatOllama(model='gemma3:1b')
for chunk in model.stream('wassup'):
    for char in chunk.content:
        for c in char:
            print(c, end="")

Not much! Just hanging out, ready to chat. 😊 

What's up with *you*? How’s your day going?

## Basics

In [90]:
from langgraph.graph import START, END, StateGraph, MessagesState
from typing import TypedDict, Literal, List, Annotated
import operator
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [15]:
class StateOne(TypedDict):
    a: int
    b: int
    answer: int

In [ ]:
def add(state):
    return ({'answer': state['a']+state['b']})

In [16]:
builder = StateGraph(StateOne)

builder.add_node("add", add)

builder.add_edge(START, "add")
builder.add_edge('add', END)

graph = builder.compile()
graph.invoke({"a":1, "b":2})

{'a': 1, 'b': 2, 'answer': 3}

## Conditional Edges

In [17]:
class StateOne(TypedDict):
    a: int
    b: int
    answer: int

In [20]:
def add(state):
    return ({'answer': state['a']+state['b']})

def sub(state):
    return ({'answer': state['a']-state['b']})

def condition(state) -> Literal['add', 'sub']:
    if state['a'] > 5:
        return "sub"
    else:
        return "add"

In [22]:
builder = StateGraph(StateOne)

builder.add_node("add", add)
builder.add_node("sub", sub)

builder.add_conditional_edges(START, condition)
builder.add_edge("sub", END)
builder.add_edge('add', END)

graph = builder.compile()

In [23]:
graph.invoke({'a':0, 'b':1})

{'a': 0, 'b': 1, 'answer': 1}

In [24]:
graph.invoke({'a':5, 'b':1})

{'a': 5, 'b': 1, 'answer': 6}

In [25]:
graph.invoke({'a':6, 'b':1})

{'a': 6, 'b': 1, 'answer': 5}

## Parallels

In [59]:
class StateOne(TypedDict):
    a: str
    answer: str

In [60]:
def add(state):
    return ({'a': state['a']+" Node 1"})

def sub(state):
    return ({'new': state['answer']+" Node 2"})
"""
def condition(state) -> Literal['node1', 'node2']:
    if state['a'] > 5:
        return "sub"
    else:
        return "add"""

'\ndef condition(state) -> Literal[\'node1\', \'node2\']:\n    if state[\'a\'] > 5:\n        return "sub"\n    else:\n        return "add'

In [61]:
builder = StateGraph(StateOne)

builder.add_node("add", add)
builder.add_node("sub", sub)

builder.add_edge(START, "add")
builder.add_edge(START, "sub")
builder.add_edge("sub", END)
builder.add_edge('add', END)

graph = builder.compile()

In [64]:
graph.invoke({'a':"hello", "answer": "lol"})

{'a': 'hello Node 1', 'answer': 'lol'}

**Each Parallel Node** can update induvidual / different keys, but to update the same key - Reducer needed.

**If a key which is not in the state** is updated, the end state **DOES NOT** include it.

### Reducers for parallel

In [ ]:
class StateOne(TypedDict):
    answer: Annotated[list[int], operator.add]

In [75]:
def node_1(state):
    print("---Node 1---")
    return {"answer": [state['answer'][0] + 1]}


builder = StateGraph(StateOne)
builder.add_node("node_1", node_1)

# Logic
builder.add_edge(START, "node_1")
builder.add_edge("node_1", END)

# Add
graph = builder.compile()

In [77]:
graph.invoke({'answer': [1]})

---Node 1---


{'answer': [1, 2]}

In [81]:
def node_1(state):
    print("---Node 1---")
    return {"answer": [state['answer'][-1] + 1]}

def node_2(state):
    print("---Node 2---")
    return {"answer": [state['answer'][-1] + 1]}

def node_3(state):
    print("---Node 3---")
    return {"answer": [state['answer'][-1] + 10]}

builder = StateGraph(StateOne)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# Logic
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_1", "node_3")
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)


# Add
graph = builder.compile()

In [82]:
graph.invoke({'answer':[0]})

---Node 1---
---Node 2---
---Node 3---


{'answer': [0, 1, 2, 11]}

## Human In The Loop

In [107]:
class State(MessagesState):
    answers: Annotated[List[Literal["Yes", "No"]], operator.add]

In [88]:
model = ChatOllama(model='gemma3:1b')

In [108]:
from pydantic import BaseModel
from typing import List, Literal

class Answer(BaseModel):
    response: Literal["Yes", "No"]

llm_structured = model.with_structured_output(Answer)

In [ ]:
prompt = """You are given a query / question. If you understand the query and know about it, Reply with "Yes". If you don't have full clarity on the query, or if you don't know about the topic asked, simply return a "No".
    """


llm_structured.invoke([])

Answer(response='Yes')

In [123]:
builder = StateGraph(State)

builder.add_node('assistant', assistant)
builder.add_edge(START, "assistant")
builder.add_edge("assistant", END)

graph = builder.compile()

In [118]:
graph.invoke({'messages': HumanMessage(content="Do you know about jldfjs")})

{'messages': [HumanMessage(content='Do you know about jldfjs', additional_kwargs={}, response_metadata={}, id='9d9aa426-d62e-42f0-8cd0-14467fb5f436'),
  HumanMessage(content='No', additional_kwargs={}, response_metadata={}, id='6e984e4b-2ad4-4e1b-a629-ad860f3ec1c8')],
 'answers': []}

In [127]:
from pydantic import BaseModel
from typing import List, Literal

class Answer(BaseModel):
    response: Literal["Yes", "No"]

llm_structured = model.with_structured_output(Answer)

In [128]:
class State(MessagesState):
    answers: Annotated[List[Literal["Yes", "No"]], operator.add]

In [140]:
def assistant(state):
    prompt = """You are given a query / question. If you understand the query and know about it, Reply with "Yes". If you don't have full clarity on the query, or if you don't know about the topic asked, simply return a "No".
    """
    answer = llm_structured.invoke([SystemMessage(content=prompt), state['messages'][-1]])
    return ({'answers': [answer.response]})

def check(state) -> Literal['__end__', 'fallback']:
    if(state['answers'][-1] == 'Yes'):
        return "__end__"
    elif(state['answers'][-1] == 'No'):
        return 'fallback'
    
def fallback(state):
    user_input = input(f"Could you please rephrase your question '{state['messages'][-1].content}'")
    return ({'messages': HumanMessage(content=user_input)})

In [141]:
builder = StateGraph(State)

builder.add_node('assistant', assistant)
builder.add_node('fallback', fallback)

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", check)
builder.add_edge("fallback", "assistant")

graph = builder.compile()

In [142]:
graph.invoke({'messages': HumanMessage(content="Do you know about jldfjs")})

{'messages': [HumanMessage(content='Do you know about jldfjs', additional_kwargs={}, response_metadata={}, id='db71945c-58bd-4562-979e-0e83f529e991'),
  HumanMessage(content='I mean Dark Matter', additional_kwargs={}, response_metadata={}, id='95175362-4121-443b-8671-f1865a8993d6')],
 'answers': ['No', 'Yes']}

In [138]:
graph.invoke({'messages': HumanMessage(content="Do you know about jldfjs")})

{'messages': [HumanMessage(content='Do you know about jldfjs', additional_kwargs={}, response_metadata={}, id='ffeffe4d-9914-4f71-ba4c-5f2c1c332815'),
  HumanMessage(content='Do you know about dark matter', additional_kwargs={}, response_metadata={}, id='59b23813-592a-4afe-8ed1-46802192b731')],
 'answers': ['No', 'Yes']}

In [139]:
graph.invoke({'messages': HumanMessage(content="Do you know about Dark matter")})

{'messages': [HumanMessage(content='Do you know about Dark matter', additional_kwargs={}, response_metadata={}, id='a2f7c70e-a230-41bd-acaa-28dcdbaaaba1')],
 'answers': ['Yes']}

In [144]:
graph.invoke({'messages': HumanMessage(content="Do you know about modified newtonian dynamics")})

{'messages': [HumanMessage(content='Do you know about modified newtonian dynamics', additional_kwargs={}, response_metadata={}, id='5c07924e-8296-48c3-a34a-848e9b6acfe6'),
  HumanMessage(content='Mond is the alternative for Dark matter, Modified newtonian dynamics', additional_kwargs={}, response_metadata={}, id='44e80bef-7894-4443-b002-b81030ba0f15'),
  HumanMessage(content='Mond', additional_kwargs={}, response_metadata={}, id='ec4938d5-549a-4ad2-906b-96286093ac54')],
 'answers': ['No', 'No', 'Yes']}